# PointClouds Experiments

This notebook shows inference experiments with VAE for point cloud compression.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px

In [ ]:
point_cloud_example_path = #...

In [ ]:
file = open(point_cloud_example_path, 'rb')
data = np.load(file)
cloud_2d = data['kitti_cloud_2d'][:,:,0:3]
print(cloud_2d.shape)
print(type(cloud_2d))
print(cloud_2d.max())
print(cloud_2d.min())

In [ ]:
print(f"Min:{cloud_2d.min()}")
print(f"Max:{cloud_2d.max()}")

In [ ]:
xs = cloud_2d[:,:,0].flatten()
ys = cloud_2d[:,:,1].flatten()
zs = cloud_2d[:,:,2].flatten()
c = np.sqrt(xs ** 2 + ys ** 2 + zs ** 2)

df = px.data.iris()
fig = px.scatter_3d(x=xs, y=ys, z=zs, color=c)
fig.update_traces(marker_size=1)
fig.show()

In [ ]:
cloud_2d.shape

# Evaluation

In [ ]:
from torchvision import transforms
from compressai.zoo import bmshj2018_hyperprior
import torch

In [ ]:
model_path = #..

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = bmshj2018_hyperprior(quality=5, pretrained=False)
net.load_state_dict(torch.load(model_path))
net.eval()
net.to(device)
print(f'Parameters: {sum(p.numel() for p in net.parameters())}')

In [ ]:
cloud_2d_resized = np.resize(cloud_2d, (64,512,3))

In [ ]:
input_tensor = transforms.ToTensor()(cloud_2d_resized).unsqueeze(0).to(device, dtype=torch.float)

with torch.no_grad():
    output_net = net.forward(input_tensor)
output_net['x_hat']#.clamp_(0, 1)
output_tensor = output_net['x_hat']

output_cloud_2d = np.asarray(output_tensor.squeeze().cpu())
recon_swapped = np.swapaxes(np.swapaxes(output_cloud_2d, 0, 2), 0,1)

In [ ]:
plt.hist(recon_swapped.flatten())

In [ ]:
xs = recon_swapped[:,:,0].flatten()
ys = recon_swapped[:,:,1].flatten()
zs = recon_swapped[:,:,2].flatten()
c = np.sqrt(xs ** 2 + ys ** 2 + zs ** 2)

df = px.data.iris()
fig = px.scatter_3d(x=xs, y=ys, z=zs, color=c)
fig.update_traces(marker_size=1)
fig.show()